<a href="https://colab.research.google.com/github/Small-Molecule-Discovery-Center/smdc_preprocess_data/blob/main/preprocess_pampa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocess PAMPA data
### How to use this notebook:
1. File --> save a copy in drive; open that copy to interact with notebook
1. Click folder icon on left side
2. Drag PAMPA file into folder space
3. Right click on PAMPA file and hit copy path
4. Paste file path into second cell
5. Edit source_plate and donor_well_conc variables if necessary
3. Run all the cells (shift+enter or click the arrows)
7. DOWNLOAD the results_processed file - it will not get saved to your system it will just be deleted if you close the tab

In [1]:
! git clone https://github.com/Small-Molecule-Discovery-Center/smdc_preprocess_data.git
import sys
sys.path.append('/content/smdc_preprocess_data')
import preprocess_pampa as pp

Cloning into 'smdc_preprocess_data'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 47 (delta 18), reused 26 (delta 7), pack-reused 0
Receiving objects: 100% (47/47), 64.00 KiB | 4.92 MiB/s, done.
Resolving deltas: 100% (18/18), done.


ModuleNotFoundError: ignored

### Variables to fill out for each plate run

In [6]:
# path to file
infile="/content/smdc_preprocess_data/data/pampa_explorer_example_results.xlsx"

# source plate barcode: if multiple, separate with semicolons like "OTPARP1-MA01;OTPARP2-MA01"
source_plate="OTPARP1-MA01"

# concentration in uM
donor_well_conc=50

### Run code

In [7]:
pp.preprocess_pampa(infile=infile)

Destination plate: pampa_explorer_example
Unique samples pre: 33
Unique samples post: 32
Data saved to ./pampa_explorer_example_processed.csv
